## Analyzing Historical Visitation Numbers of U.S. National Parks
https://github.com/carl-schick-ds/national-parks

***

This project will analyze the historical visitation counts for each U.S. National Park and attempt to answer the following questions: 
 - How does the age of a national park impact annual visitation?
 - How does the area-size of a national park impact annual visitation?
 - Are visitation numbers impacted when a site receives an official national park designation?
 - Does the location of a national park impact annual visitation?
 - How does the weather impact monthly visitation?

***

### Data Collection
The first step of the project is to collect the data from various sources on the internet.  

Data is currently collected from two main sources...
 - The National Park Services' Integrated Resourcde Management Applications (IRMA) Portal
   - List of official national park unit codes
   - Monthly visitation counts for each national park
 - Wikipedia
   - Location (lat/long), date established, and gross area acres for each national park

Web scraping is performed using the [Beautfiul Soup](https://www.crummy.com/software/BeautifulSoup/) library.

**_Data collection will rebuild/refresh the CSV files used for the remainder of the notebook._**  
**_Data collection can be toggled on/off with the REFRESH_DATA boolean flag._**

In [27]:
REFRESH_DATA = True

import numpy as np
import pandas as pd
import os

if REFRESH_DATA:
    from bs4 import BeautifulSoup
    import requests
    import time
    import re
    user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'
    headers = {'User-Agent': user_agent}

In [28]:
if REFRESH_DATA:
    from requests.adapters import HTTPAdapter
    from requests.packages.urllib3.util.retry import Retry
    retry_strategy = Retry(
        total=3,
        backoff_factor=1,
        status_forcelist=[429, 500, 502, 503, 504],
        allowed_methods=["HEAD", "GET", "OPTIONS"]
    )
    adapter = HTTPAdapter(max_retries=retry_strategy)
    http = requests.Session()
    http.mount("https://", adapter)
    http.mount("http://", adapter)

In [29]:
if REFRESH_DATA:

    park_units_url = 'https://irmaservices.nps.gov/v2/rest/unit/designations'
    park_units_namespace = {'root': 'NRPC.IrmaServices.Rest.Unit'}
    park_unit_exceptions = {'DENG':'DENA', 'GAAG':'GAAR', 'GLBG':'GLBA', 'GRDG':'GRSA', 'KATG':'KATM', 'LACG':'LACL', 'WRSG':'WRST'}
    
    national_parks = {}
    r = http.get(park_units_url, headers=headers)
    soup = BeautifulSoup(r.text, "xml")
    # print(soup)

    for unit_designation in soup.find_all('UnitDesignation'):
        if unit_designation.find("Code").text == 'NP':
            units = unit_designation.find("Units")
            for value in units.find_all('Value'):
                raw_code = value.find("Code").text
                name = value.find("Name").text
                code = raw_code if raw_code not in park_unit_exceptions.keys() else park_unit_exceptions[raw_code]
                national_parks[code] = name

    # print(national_parks)
    park_units_df = pd.DataFrame.from_dict(national_parks, orient='index', columns=['name'])
    park_units_df.index.name = 'code'
    park_units_df.to_csv('national_park_units.csv')
    # print(park_units_df.head())

In [30]:
if REFRESH_DATA:

    # park_visits_homepage = 'https://irma.nps.gov/STATS/'
    park_visits_domain = 'https://irma.nps.gov'
    park_visits_url = '/STATS/SSRSReports/Park%20Specific%20Reports/Recreation%20Visitors%20By%20Month%20(1979%20-%20Last%20Calendar%20Year)'
    park_visits_qs = '?Park='
    park_visits_df = pd.DataFrame()
    target_table_min = 10

    print('Processing:', end=" ")
    for park_code in park_units_df.index:
        print(park_code, end=", ")
        park_visits_request = park_visits_domain + park_visits_url + park_visits_qs + park_code
        r = http.get(park_visits_request, headers=headers, timeout=5)
        soup = BeautifulSoup(r.text, "html")
        park_visits_iframe = soup.find('iframe').attrs['src']

        park_visits_request = park_visits_domain + park_visits_iframe
        r = http.get(park_visits_request, headers=headers, timeout=5)

        dfs = pd.read_html(r.text, match="Year", skiprows=1)
        for df in dfs:
            if len(df) > target_table_min: one_park_df = df
        
        new_header = one_park_df.iloc[0] #grab the first row for the header
        one_park_df = one_park_df[1:] #take the data less the header row
        one_park_df.columns = new_header #set the header row as the df header
    
        one_park_df = one_park_df.fillna(0)
        if 'Total' in one_park_df.columns:
            one_park_df.drop('Total', axis=1, inplace=True)

        one_park_df.set_index('Year', inplace=True)
        one_park_srs = one_park_df.stack()

        park_visits_df[park_code] = one_park_srs

    park_visits_df = park_visits_df.fillna(0)
    park_visits_df.index.names = ['Year', 'Month']
    park_visits_df.to_csv('national_park_visits.csv')


Processing: ACAD, ARCH, BADL, BIBE, BISC, BLCA, BRCA, CANY, CARE, CAVE, CHIS, CONG, CRLA, CUVA, DENA, DEVA, DRTO, EVER, GAAR, GLAC, GLBA, GRBA, GRCA, GRSA, GRSM, GRTE, GUMO, HALE, HAVO, HOSP, INDU, ISRO, JEFF, JOTR, KATM, KEFJ, KICA, KOVA, LACL, LAVO, MACA, MEVE, MORA, NOCA, NPSA, OLYM, PEFO, PINN, REDW, ROMO, SAGU, SEQU, SHEN, THRO, VIIS, VOYA, WHSA, WICA, WRST, YELL, YOSE, ZION, 

In [47]:
if REFRESH_DATA:

    park_data_url = 'https://en.wikipedia.org/wiki/List_of_national_parks_of_the_United_States'
    
    r = http.get(park_data_url, headers=headers)
    soup = BeautifulSoup(r.text, "xml")
    # print(soup)

    dfs = pd.read_html(r.text, match="Date established as park")
    park_data_df = dfs[0]

    # Strip years and footnotes from columns
    for column_name in park_data_df.columns.values:
        if '(' in column_name or '[' in column_name:
            new_column_name = re.sub("\(.*?\)","", column_name)
            new_column_name = re.sub("\[.*?\]","", new_column_name)
            new_column_name = new_column_name.strip()
            park_data_df.rename(columns={column_name: new_column_name}, inplace=True)
 
    # Rename some columns and drop unneeded columns
    park_data_df.rename(columns={'Date established as park': 'Established', 'Area': 'Acres'}, inplace=True)
    park_data_df.drop(columns=['Image', 'Recreation visitors'], inplace=True)

    # Parse state and lat/long from Location into new columns; then drop Location
    park_data_df['State'] = park_data_df['Location'].apply(lambda x: re.split('[^a-zA-Z\s]', x)[0])
    park_data_df['Lat-Long'] = park_data_df['Location'].apply(lambda x: x.rpartition('/')[2].strip())
    park_data_df.drop(columns=['Location'], inplace=True)

    # Clean Acres column to float
    park_data_df['Acres'] = park_data_df['Acres'].apply(lambda x: re.split('[0-9,.]', x)[0])

    print(park_data_df.head())

    # print(national_parks)
    # national_parks_df = pd.DataFrame.from_dict(national_parks, orient='index', columns=['name'])
    # national_parks_df.index.name = 'code'
    # national_parks_df.to_csv('national_parks.csv')
    # print(national_parks_df.head())

             Name        Established Acres  \
0          Acadia  February 26, 1919         
1  American Samoa   October 31, 1988         
2          Arches  November 12, 1971         
3        Badlands  November 10, 1978         
4        Big Bend      June 12, 1944         

                                         Description           State  \
0  Covering most of Mount Desert Island and other...           Maine   
1  The southernmost national park is on three Sam...  American Samoa   
2  This site features more than 2,000 natural san...            Utah   
3  The Badlands are a collection of buttes, pinna...    South Dakota   
4  Named for the prominent bend in the Rio Grande...           Texas   

            Lat-Long  
0   ﻿44.35°N 68.21°W  
1  ﻿14.25°S 170.68°W  
2  ﻿38.68°N 109.57°W  
3  ﻿43.75°N 102.50°W  
4  ﻿29.25°N 103.25°W  


In [32]:
park_visits_df = pd.read_csv('national_park_visits.csv', index_col=[0,1])
print(park_visits_df.head())

              ACAD    ARCH    BADL   BIBE   BISC   BLCA    BRCA    CANY  \
Year Month                                                                
2021 JAN     20268   48725   14899  50298  25626   8759   37725   18837   
     FEB     16123   53986   11350  43889  19596   7287   35653   20164   
     MAR     38060  151074   28182  89915  30655   7236   96800   87001   
     APR    110096  193912   42749  67628  30835  14159  201771  116971   
     MAY    324654  225789  109764  53067  37128  34729  297905  134778   

              CARE   CAVE  ...    SHEN   THRO   VIIS   VOYA   WHSA   WICA  \
Year Month                 ...                                              
2021 JAN     20786  16219  ...   37208   6866  18736   4679  45053  14840   
     FEB     25718  13270  ...    9771   5162  19680   7887  35766  10836   
     MAR    115728  30749  ...   75294  15125  22302   1558  98118  16430   
     APR    165498  25859  ...  114583  11993  26531    104  78723  43241   
     MAY    